# architecture-17 ( Basic Classification )

## What's new:

1- Add


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import Input
import talib

# Constants
SEQUENCE_LENGTH = 60
FORECAST_HORIZON = 10


In [ ]:
# Load dataset
df = pd.read_csv('datasets-17/XAGUSD-H1-rates.csv', sep='\t').dropna()

In [6]:
df.head()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2015.01.02,09:00:00,15.889,15.896,15.807,15.834,993,18451000,21
1,2015.01.02,10:00:00,15.836,15.864,15.822,15.834,1116,25764000,22
2,2015.01.02,11:00:00,15.836,15.838,15.790,15.822,1023,18885000,19
3,2015.01.02,12:00:00,15.827,15.828,15.725,15.772,1286,23593000,21
4,2015.01.02,13:00:00,15.781,15.786,15.667,15.734,1584,28750000,17


In [18]:
# Scaling Features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']])

In [21]:
# Add RSI for post-processing
df['RSI'] = talib.RSI(df['<CLOSE>'], timeperiod=14)

# Label trend reversals (example: a simplistic method)
def label_trend_reversals(prices, window=5):
    labels = []
    for i in range(len(prices) - window):
        past = prices[i:i+window//2]
        future = prices[i+window//2:i+window]
        if np.mean(future) > np.mean(past):
            labels.append(1)  # potential buy
        elif np.mean(future) < np.mean(past):
            labels.append(2)  # potential sell
        else:
            labels.append(0)
    labels += [0] * window  # pad
    return labels

df['Label'] = label_trend_reversals(df['<CLOSE>'].values)


In [22]:
# Prepare Sequences
X, y = [], []
for i in range(SEQUENCE_LENGTH, len(scaled_data) - FORECAST_HORIZON):
    X.append(scaled_data[i-SEQUENCE_LENGTH:i])
    y.append(df['Label'].iloc[i:i+FORECAST_HORIZON].values)

X, y = np.array(X), np.array(y)


In [23]:
# Split Dataset
X_train = X[:-1]
y_train = y[:-1]

# Last 60 candles for prediction
X_input = X[-1:]
y_true_future = y[-1]  # for evaluation if desired


In [31]:
# Build LSTM Classification Model
model = Sequential([
    Input(shape=(SEQUENCE_LENGTH, X.shape[2])),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(FORECAST_HORIZON * 3, activation='softmax')  # 3 classes per forecast hour
])

In [32]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape labels for categorical classification per hour
y_train_reshaped = y_train.reshape(-1, FORECAST_HORIZON)


In [33]:
# Train the Model
model.fit(X_train, y_train_reshaped, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 10), output.shape=(None, 30)

In [ ]:
# Predict Next 10 Hours
pred_probs = model.predict(X_input)
pred_classes = np.argmax(pred_probs.reshape(FORECAST_HORIZON, 3), axis=1)

# Apply RSI filter
last_60_index = len(df) - FORECAST_HORIZON - 1
rsi_segment = df['RSI'].iloc[last_60_index+1:last_60_index+1+FORECAST_HORIZON].values
final_preds = []

for i, (pred, rsi) in enumerate(zip(pred_classes, rsi_segment)):
    if pred == 1 and rsi < 30:
        final_preds.append(1)  # buy
    elif pred == 2 and rsi > 70:
        final_preds.append(2)  # sell
    else:
        final_preds.append(0)


In [ ]:
# Visualize Forecast
import matplotlib.dates as mdates

# Extract last 4 real candles
recent_candles = df.iloc[-FORECAST_HORIZON-4:-FORECAST_HORIZON]

# Fake future time axis
future_times = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])
future_start = future_times.iloc[-FORECAST_HORIZON-1]
future_time_axis = [future_start + pd.Timedelta(hours=i+1) for i in range(FORECAST_HORIZON)]

# Plot
plt.figure(figsize=(12, 6))

# Plot the last 4 candles
plt.plot(future_times.iloc[-FORECAST_HORIZON-4:-FORECAST_HORIZON],
         recent_candles['CLOSE'], label='Last 4 Candles', color='blue')

# Vertical dotted line between real and forecast
plt.axvline(future_time_axis[0], linestyle='--', color='black', label='Forecast Start')

# Add vertical lines for predictions
for i, pred in enumerate(final_preds):
    if pred == 1:
        label = 'Buy'
        color = 'green'
    elif pred == 2:
        label = 'Sell'
        color = 'red'
    else:
        continue
    plt.axvline(future_time_axis[i], linestyle='--', color=color)
    plt.text(future_time_axis[i], plt.ylim()[1]*0.98, str(pred), color=color, fontsize=10, ha='center')

plt.xlabel('Time')
plt.ylabel('Price (CLOSE)')
plt.title('Forex Trend Reversal Predictions')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
